# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [4]:
# number of callbacks for black-sounding names
sum(data[data['race']=='w']['call'])

235.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

In [9]:
w = data[data['race']=='w']
b = data[data['race']=='b']
w_call = w['call']
b_call = b['call']

### 1. What test is appropriate for this problem? Does CLT apply?

Since we're working with a Bernoulli distribution taken from a sample of the true population, it'd be best if we applied the t-test for this problem to approximate the difference in callbacks due to race.

The Central Limit Theorem should apply here as we are working with 2435 datapoints per race category. 

### 2. What are the null and alternate hypotheses?

The null hypothesis (Ho) is that there is no statistically significant difference between the callbacks for the black-sounding names and the white-sounding names.

The alternate hypothesis (Ha) is that there is a statistically significant difference between the call backs for the black-sounding names and the white-sounding names.

### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [10]:
# Doing our tests on the original data first.
perc_w_calls = sum(w_call) / len(w)
perc_b_calls = sum(b_call) / len(w)
perc_diff = (perc_w_calls - perc_b_calls) / perc_b_calls

print ('Percent callbacks for white-sounding names: {:.2%}'.format(perc_w_calls))
print ('Percent callbacks for black-sounding names: {:.2%}'.format(perc_b_calls))
print ('Difference in callbacks: {:.2%}'.format(perc_w_calls - perc_b_calls))
print ('Resumes with white-sounding names received {:.2%} more callbacks than resumes with black-sounding names.'.format(perc_diff))

Percent callbacks for white-sounding names: 9.65%
Percent callbacks for black-sounding names: 6.45%
Difference in callbacks: 3.20%
Resumes with white-sounding names received 49.68% more callbacks than resumes with black-sounding names.


In [11]:
# Recall that datasets with Bernoulli distributions have their probability of
# occuring at the sample mean.

w_call.describe()

count    2435.000000
mean        0.096509
std         0.295346
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: call, dtype: float64

In our original dataset, resumes with white-sounding names received 3.2% more callbacks with identical resumes.

We'll start applying bootstrapping techniques and the frequentist statsitical approach at this point.

In [12]:
print (len(b_call))

2435


In [13]:
print (stats.ttest_ind(w_call, b_call))

Ttest_indResult(statistic=4.114705290861751, pvalue=3.940802103128886e-05)


The provided t-test function from scipy.stats suggests that the callbacks for the two races are significantly different.

In [18]:
# First we'll check if the difference is significant. (alpha = 0.05)
empirical_diff = np.mean(w_call) - np.mean(b_call)

concat_data = np.concatenate((b_call, w_call))

permutated_repl = np.empty(10000)

for i in range(10000):
    permutated_data = np.random.permutation(concat_data)
    b_callbacks = permutated_data[:len(b)]
    w_callbacks = permutated_data[len(b):]
    permutated_repl[i] = abs(np.mean(w_callbacks) - np.mean(b_callbacks))

p_value = sum(permutated_repl >= empirical_diff) / len(permutated_repl)
print ('p-value:', p_value)


p-value: 0.0001


In 10,000 trials of the null hypothesis, the difference in mean that extreme occured 1/10000 times.

The p-value of 0.0001 (< 0.05) supports that the difference between the callbacks for the black-sounding names and white-sounding names samples are statistically significant.

In [20]:
new_b_data_repl = np.empty(10000)
new_w_data_repl = np.empty(10000)
diff_of_mean_repl = np.empty(10000)

for i in range(10000):
    new_b_data_repl[i] = np.mean(np.random.choice(b_call, len(b_call)))
    new_w_data_repl[i] = np.mean(np.random.choice(w_call, len(w_call)))
    diff_of_mean_repl[i] = new_w_data_repl[i] - new_b_data_repl[i]
    
confidence_interval_b = np.percentile(new_b_data_repl, [2.5, 97.5])
confidence_interval_w = np.percentile(new_w_data_repl, [2.5, 97.5])
confidence_interval_diff = np.percentile(diff_of_mean_repl, [2.5, 97.5])
total_diff = sum(diff_of_mean_repl >= empirical_diff) / len(diff_of_mean_repl)
print ('95% Confidence Interval for Black Race:', confidence_interval_b)
print ('95% Confidence interval for White Race:', confidence_interval_w)
print ('95% Confidence Interval for Difference:', confidence_interval_diff)
print ('Occurence greater or equal to empirical diff:', total_diff)

95% Confidence Interval for Black Race: [0.0550308  0.07433265]
95% Confidence interval for White Race: [0.08501027 0.10841889]
95% Confidence Interval for Difference: [0.01683778 0.0476386 ]
Occurence greater or equal to empirical diff: 0.5037


If the data for each respective race is resampled 10,000, we can expect the difference in means to be statistically significant about 50% of the time.

### 4. Write a story describing the statistical significance in the context or the original problem.

<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

The pervasiveness of racial discrimination includes, but is not limited to, the US job market. Using 5000 unique resumes to apply for 1300 jobs, we see a discrepancy between the callbacks received by names associated with colored (black) and non-colored (white) races. Despite using identical for resumes for both races, we observed that resumes associated with white names received approximately 50% more callbacks than resumes with black names (6% vs. 3%). 

To determine if the difference was statistically signicant, we utilized a basic t-test for the two original samples, bootstrap technique, and the frequentist approach. We set the significance cutoff (alpha) at 0.05. With the basic t-test, we observed a p-value of 3.94E-5, which is lower than 0.05. With the bootstrap technique, we observed a p-value of 0.001 which suggests that, assuming that there is no difference between the two races, we would only see a incident with that extreme of a difference 1/10000 trials. The results for the frequentist approach shows that if we resampled the data for each respective race 10000 times, we would observe a difference that extreme or more in at 50% of the trials.

All test approaches provide statistical significance that racial discrimination has pervaded into the US job market and white individuals on average receive more callbacks than black individuals.

### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis only addresses the difference in callbacks between white and black individuals, all other factors fixed. This does not mean that race/name is the most important factor in callback success because we did not study callbacks across resume characters such as years of experience and education. To determine the most important factors in callback success, we would need to determine the correlation coefficient between callbacks and all other column. For there, we can select the columns with the highest value to study further.